In [3]:
import os
if 'Conformal-Sparsemax/notebooks' in os.getcwd():
    os.chdir(os.path.dirname(os.getcwd()))

In [4]:
from conformal_sparsemax.classifier import CNN,get_data,train,evaluate

/home/joaocalem/.pyenv/versions/3.10.6/envs/stable-diffusion/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/joaocalem/.pyenv/versions/3.10.6/envs/stable-diffusion/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
import torch
from torch import nn
from sklearn.metrics import f1_score

In [6]:
from entmax.losses import SparsemaxLoss, Entmax15Loss

In [7]:
loss = 'softmax'
dataset = 'CIFAR100' #CIFAR100 or MNIST

if loss == 'sparsemax':
    criterion = SparsemaxLoss()
elif loss == 'entmax15':
    criterion = Entmax15Loss()
elif loss == 'softmax':
    criterion = torch.nn.NLLLoss()
train_dataloader, dev_dataloader, test_dataloader, _ = get_data(0.2,16,dataset = dataset)

if dataset == 'CIFAR100':
    model = CNN(loss)
elif dataset == 'MNIST':
    model = CNN(loss,n_classes=10,input_size=28,channels=1)
else:
    raise Exception('Wrong dataset name')
    
model, train_history, val_history, f1_history = train(model,
                                            train_dataloader,
                                            dev_dataloader,
                                            criterion,
                                            epochs=50,
                                            patience=3)

_, predicted_labels, true_labels, test_loss = evaluate(
                                                    model,
                                                    test_dataloader,
                                                    criterion)

f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Test loss: {test_loss:.3f}')
print(f'Test f1: {f1:.3f}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169001437/169001437 [00:15<00:00, 10965507.55it/s]


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified
-- Epoch 1 --


2450it [00:10, 243.01it/s]


train_loss: -0.010
val_loss: -0.010
val_f1: 0.001
-- Epoch 2 --


2253it [00:08, 257.59it/s]


KeyboardInterrupt: 

In [ ]:
results = {
    'train_history':train_history,
    'val_history':val_history,
    'f1_history':f1_history,
}

In [ ]:
import json
with open(f'results/{dataset}_{loss}_results.json', 'w') as f:
    json.dump(results, f)

In [ ]:
torch.save(model.state_dict(), f'models/{dataset}_{loss}.pth')